In [2]:
!pip install zernpy
import matplotlib.pyplot as plt
import numpy as np
import cv2
import pandas as pd
import zernpy
from zernpy import ZernPol
from zernpy import generate_polynomials, fit_polynomials, generate_random_phases, generate_phases_image, fit_polynomials_vectors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 6.8 MB/s eta 0:00:00


In [ ]:
#Creating a mask

# Get image dimensions
height, width = (420,420)

# Calculate center coordinates
center_x = width // 2
center_y = height // 2

# Define radius of the circular mask
radius = min(center_x, center_y)  # Use the smaller dimension to ensure the circle fits

# Create a meshgrid of coordinates
Y, X = np.ogrid[:height, :width]

# Calculate distance from the center for each point
dist_from_center = np.sqrt((X - center_x)**2 + (Y - center_y)**2)

# Create the circular mask
mask = dist_from_center <= radius

In [ ]:
def rms(data):
    return np.sqrt(np.mean((data - np.mean(data))**2))

In [ ]:
from google.colab import files
h = []
rms_value=[]
for i in range(1,25,1):
  h.append(ZernPol(noll_index=i))
zernike_cof = np.empty((1000,24)) # 25 set of Zernike Polynomials to be extracted
for i in range(314,1000,1):
  f = np.load(f"frame_{i:04d}.npy", allow_pickle=True) # loading the wavefront frame saved in .npy format
  R, Theta = ZernPol.gen_equal_polar_mesh(n_points=176400) # creating a polar coordinates mesh
  # using function to fit polynomials to the wavefront frame
  co1,_ = fit_polynomials(
	  phases_image=f,
	  polynomials=h,
	  crop_radius=1.0,
	  suppress_warnings= False,
	  strict_circle_border= True,
	  round_digits= 4,
	  return_cropped_image= False
  )
  zernike_cof[i,:]=co1    #saving the coefficients
  phase = generate_phases_image(polynomials=h, polynomials_amplitudes=co1, img_width=420, img_height=420)  #generating the image from zernike coefficients
  residue = phase - f*(mask) #subtrating the original wavefront with zernike projected image to find the residue
  rms_value.append(rms(residue))
  if i%100==0:
    print(i, "iterations completed")
df = pd.DataFrame(zernike_cof)
#saving zernike coefficients to excel file
df.to_excel("zernike_coefficients_1000.xlsx", index=True, header=True, engine="openpyxl")
files.download("zernike_coefficients_1000.xlsx")
#saving the phase_values to another excel file
df2 = pd.DataFrame(rms_value)
df2.to_excel("rms_phase.xlsx", index=True, header=True, engine="openpyxl")
files.download("rms_phase.xlsx")

In [ ]:
# Plotting the Zernike_fitted wavefront against original wavefront

phase = generate_phases_image(polynomials=h, polynomials_amplitudes=co1, img_width=420, img_height=420)

vmin = min(phase.min(), f.min())
vmax = max(phase.max(), f.max())

# 10. Plot the Original and Fitted Wavefronts with a common color scale
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

# Plot original wavefront
im1 = ax[0].imshow(phase, extent=[0, 400, 0, 400], cmap='jet', vmin=vmin, vmax=vmax)
ax[0].set_title("Zernike-Fitted Wavefront")
ax[0].set_xlabel("X")
ax[0].set_ylabel("Y")

# Plot fitted wavefront
im2 = ax[1].imshow(f*mask, extent=[0, 400, 0, 400], cmap='jet', vmin=vmin, vmax=vmax)
ax[1].set_title("Original wavefront")
ax[1].set_xlabel("X")
ax[1].set_ylabel("Y")

# Add a single shared colorbar
cbar = fig.colorbar(im2, ax=ax, orientation="vertical", fraction=0.05, pad=0.02)
cbar.set_label("Wavefront Phase (rad)")
plt.show()

sum =0
for i in range(len(co1)):
  sum+= co1[i]**2
print(np.sqrt(sum + (rms(residue))**2 ))
print(rms(f*mask))